In [1]:
import numpy as np

In [2]:
m = np.random.randn(1000, 1000)

In [3]:
%%time
np.dot(m, m);

CPU times: user 231 ms, sys: 3.28 ms, total: 234 ms
Wall time: 42.3 ms


In [7]:
from numba import njit, prange
import numba as nb

@njit(parallel=True)
def dot_py(A, B):
    m, n = A.shape
    p = B.shape[1]
    C = np.zeros((m,p))
    
    for i in prange(0,m):
        for j in range(0,i):
            tmp = 0
            for k in range(0,n):
                tmp += A[i,k]*B[k,j] 
            C[i,j] = tmp
    return C

In [9]:
%%time
dot_py(m,m);

CPU times: user 667 ms, sys: 6.01 ms, total: 673 ms
Wall time: 164 ms


In [92]:
import numbers
import math

class UpperTriangular2DMatrix:
    
    def __init__(self, values):
        # len(values) = 1 / 2 * N * (N + 1)        
        N = 1 / 2 * (math.sqrt(1 + 8 * len(values)) - 1)
        if not N.is_integer():
            raise ValueError(f"The number of values, len(values)={len(values)}, do not represent an upper triangular matrix of a square matrix.")
        self.n_rows = self.n_cols = int(N)
        self.values = values
    
    @property
    def shape(self):
        return self.n_rows, self.n_cols
    
    def __xy_to_index(self, i_row, i_col):
        if i_row > i_col:
            i_row, i_col = i_col, i_row
        index = int(i_row / 2 * (2 * self.n_cols - i_row + 1) + (i_col - i_row))
        return index
    
    def __getitem_single_element(self, i_row, i_col):
        if i_row > i_col:
            i_row, i_col = i_col, i_row
        index = self.__xy_to_index(i_row, i_col)
        value = self.values[index]
        return value
    
    def __getitem_array(self, i_row, slice_col):
        if slice_col.start >= i_row:
            index_start = self.__xy_to_index(i_row, slice_col.start)
            index_stop = index_start + (slice_col.stop - slice_col.start)
            array = self.values[index_start: index_stop]
        else:
            lt_start = slice_col.start
            lt_stop = i_row
            
            

    def __getitem__(self, indices):
        if isinstance(indices, int):
            row = indices
            col = slice(0, self.n_cols)
            obj_type = "row"
            value = self.__getitem_array(row, col)
        elif isinstance(indices, tuple):
            row, col = indices
            if isinstance(row, int) and isinstance(col, int):
                obj_type = "single_element"
                value = self.__getitem_single_element(row, col)
            elif isinstance(row, int) and isinstance(col, slice):
                obj_type = "row"
                value = self.__getitem_array(row, col)
            elif isinstance(row, slice) and isinstance(col, int):
                obj_type = "column"
            elif isinstance(row, slice) and isinstance(col, slice):
                obj_type = "2d_array"
                
        return value
    
    
values = [[0, 1, 2, 3],
             [0, 1 ,2],
                [0, 1], 
                   [0]]
values = [x for row in values for x in row]
m = UpperTriangular2DMatrix(values)
print(m[0, 3], m[1, 2], m[3, 3])
print(m[3, 0], m[2, 1], m[3, 3])
print(m[2])


3 1 0
3 1 0
None
